In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Importing the libraries

In [2]:
import findspark
from elasticsearch import Elasticsearch
import requests
from pprint import pprint

### Finding locally installed Spark and importing PySpark libraries

In [3]:
findspark.init("/usr/local/spark/")
from pyspark.sql import SparkSession, functions as func

### Checking if Elasticsearch has successfully been connected

In [4]:
res = requests.get('http://localhost:9200')
pprint(res.content)

(b'{\n  "name" : "ip-172-31-14-175",\n  "cluster_name" : "elasticsearch",\n  "'
 b'cluster_uuid" : "ypf0RlCFQYytEO-JNW-MCg",\n  "version" : {\n    "number" :'
 b' "7.9.2",\n    "build_flavor" : "default",\n    "build_type" : "tar",\n    '
 b'"build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",\n    "build_da'
 b'te" : "2020-09-23T00:45:33.626720Z",\n    "build_snapshot" : false,\n    "'
 b'lucene_version" : "8.6.2",\n    "minimum_wire_compatibility_version" : "6'
 b'.8.0",\n    "minimum_index_compatibility_version" : "6.0.0-beta1"\n  },\n  '
 b'"tagline" : "You Know, for Search"\n}\n')


### Creating an Elasticsearch object

In [5]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

### Creating Spark Session

In [6]:
spark = SparkSession.builder.appName('task').getOrCreate()

### Reading CSV with Spark and creating Dataframe

In [7]:
df = spark.read.format("csv").option("header","true").load("/home/ubuntu/Hr5m.csv").fillna(0)["Emp ID", "Month Name of Joining", "Last Name", "Gender", "E Mail", "SSN", "County", "State", "Region", "City", "Zip", "Salary", func.regexp_replace(func.col("Last % Hike"), "%", "").alias("Salary Hike")]
df.show(5)

+------+---------------------+---------+------+--------------------+-----------+--------------------+-----+-------+----------+-----+------+-----------+
|Emp ID|Month Name of Joining|Last Name|Gender|              E Mail|        SSN|              County|State| Region|      City|  Zip|Salary|Salary Hike|
+------+---------------------+---------+------+--------------------+-----------+--------------------+-----+-------+----------+-----+------+-----------+
|742048|              January|   Mccoll|     F|lizeth.mccoll@ibm...|171-86-6830|               Stark|   OH|Midwest|  Alliance|44601|147446|         14|
|671135|                April|     Hern|     F|argentina.hern@nt...|083-02-3078|District of Columbia|   DC|  South|Washington|20411|129174|          8|
|965851|             December|  Patillo|     M|damian.patillo@ou...|326-11-9852|              Fresno|   CA|   West|    Burrel|93607|158746|          8|
|224660|                 June| Hagopian|     F|imogene.hagopian@...|656-36-0772|        

### Checking the total records in the PySpark Dataframe

In [8]:
print(f"Total records present: {df.count()}")

Total records present: 5000000


### Checking the data types

In [9]:
df.dtypes

[('Emp ID', 'string'),
 ('Month Name of Joining', 'string'),
 ('Last Name', 'string'),
 ('Gender', 'string'),
 ('E Mail', 'string'),
 ('SSN', 'string'),
 ('County', 'string'),
 ('State', 'string'),
 ('Region', 'string'),
 ('City', 'string'),
 ('Zip', 'string'),
 ('Salary', 'string'),
 ('Salary Hike', 'string')]

### Casting 'string' value to 'int'

In [10]:
df = df.withColumn("Emp ID", df["Emp ID"].cast("int"))
df = df.withColumn("Zip", df["Zip"].cast("int"))
df = df.withColumn("Salary", df["Salary"].cast("int"))
df = df.withColumn("Salary Hike", df["Salary Hike"].cast("int"))
df.dtypes

[('Emp ID', 'int'),
 ('Month Name of Joining', 'string'),
 ('Last Name', 'string'),
 ('Gender', 'string'),
 ('E Mail', 'string'),
 ('SSN', 'string'),
 ('County', 'string'),
 ('State', 'string'),
 ('Region', 'string'),
 ('City', 'string'),
 ('Zip', 'int'),
 ('Salary', 'int'),
 ('Salary Hike', 'int')]

### Checking if the changes have been made successfully

In [11]:
df.show(20)

+------+---------------------+---------+------+--------------------+-----------+--------------------+-----+---------+-------------+-----+------+-----------+
|Emp ID|Month Name of Joining|Last Name|Gender|              E Mail|        SSN|              County|State|   Region|         City|  Zip|Salary|Salary Hike|
+------+---------------------+---------+------+--------------------+-----------+--------------------+-----+---------+-------------+-----+------+-----------+
|742048|              January|   Mccoll|     F|lizeth.mccoll@ibm...|171-86-6830|               Stark|   OH|  Midwest|     Alliance|44601|147446|         14|
|671135|                April|     Hern|     F|argentina.hern@nt...|083-02-3078|District of Columbia|   DC|    South|   Washington|20411|129174|          8|
|965851|             December|  Patillo|     M|damian.patillo@ou...|326-11-9852|              Fresno|   CA|     West|       Burrel|93607|158746|          8|
|224660|                 June| Hagopian|     F|imogene.hag

### Checking the Schema

In [12]:
df.printSchema()

root
 |-- Emp ID: integer (nullable = true)
 |-- Month Name of Joining: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- E Mail: string (nullable = true)
 |-- SSN: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zip: integer (nullable = true)
 |-- Salary: integer (nullable = true)
 |-- Salary Hike: integer (nullable = true)



### Ingesting Pyspark Dataframe in Elasticsearch directly

In [13]:
df.write.format(
    "org.elasticsearch.spark.sql"
).option(
    "es.resource", '%s' % ('humans')
).option(
    "es.nodes", 'localhost'
).option(
    "es.port", '9200'
).save()
print("Successfully ingested data into Elasticsearch!")

Successfully ingested data into Elasticsearch!


### Setting the output limit

In [14]:
es.indices.put_settings(index="humans", body = {"index" : { "max_result_window" : 5000000 }})

{'acknowledged': True}

### Counting the records in index to see if PySpark Dataframe has been successfully ingested

In [15]:
es.count(index= "humans")

{'count': 5000000,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

### Task 1: Count the number of employees in each County, Region and City

In [16]:
es.search(index="humans", size=0, body={"query": {"match_all": {}}, "aggs":{"No. of Employees in per County" : {"terms":{"field": "County.keyword"}}}})

{'took': 153,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'No. of Employees in per County': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 4574483,
   'buckets': [{'key': 'Los Angeles', 'doc_count': 61870},
    {'key': 'Jefferson', 'doc_count': 58354},
    {'key': 'Washington', 'doc_count': 55558},
    {'key': 'Montgomery', 'doc_count': 46164},
    {'key': 'Orange', 'doc_count': 39923},
    {'key': 'Franklin', 'doc_count': 38340},
    {'key': 'Jackson', 'doc_count': 35621},
    {'key': 'District of Columbia', 'doc_count': 31760},
    {'key': 'Wayne', 'doc_count': 29634},
    {'key': 'Monroe', 'doc_count': 28293}]}}}

In [17]:
es.search(index="humans", size=0, body={"query": {"match_all": {}}, "aggs":{"No. of Employees per Region" : {"terms":{"field": "Region.keyword"}}}})

{'took': 97,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'No. of Employees per Region': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'South', 'doc_count': 1855656},
    {'key': 'Midwest', 'doc_count': 1372439},
    {'key': 'Northeast', 'doc_count': 886984},
    {'key': 'West', 'doc_count': 884921}]}}}

In [18]:
es.search(index="humans", size=0, body={"query": {"match_all": {}}, "aggs":{"No. of Employees per City" : {"terms":{"field": "City.keyword"}}}})

{'took': 249,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'No. of Employees per City': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 4820798,
   'buckets': [{'key': 'Washington', 'doc_count': 37753},
    {'key': 'Houston', 'doc_count': 23423},
    {'key': 'New York City', 'doc_count': 19153},
    {'key': 'El Paso', 'doc_count': 18438},
    {'key': 'Dallas', 'doc_count': 15789},
    {'key': 'Atlanta', 'doc_count': 14878},
    {'key': 'Springfield', 'doc_count': 13334},
    {'key': 'Sacramento', 'doc_count': 12427},
    {'key': 'Miami', 'doc_count': 12032},
    {'key': 'Los Angeles', 'doc_count': 11975}]}}}

### Task 2: Generate Employee Summary

In [19]:
es.search(index="humans", size=5, body={"query": {"match_all": {}}})

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 1.0,
  'hits': [{'_index': 'humans',
    '_type': '_doc',
    '_id': 'rgS8DnUBFUuBiHlmLycz',
    '_score': 1.0,
    '_source': {'Emp ID': 426016,
     'Month Name of Joining': 'April',
     'Last Name': 'Ardoin',
     'Gender': 'F',
     'E Mail': 'maud.ardoin@ibm.com',
     'SSN': '283-15-5867',
     'County': 'Pinellas',
     'State': 'FL',
     'Region': 'South',
     'City': 'Palm Harbor',
     'Zip': 34684,
     'Salary': 118802,
     'Salary Hike': 12}},
   {'_index': 'humans',
    '_type': '_doc',
    '_id': 'rwS8DnUBFUuBiHlmLycz',
    '_score': 1.0,
    '_source': {'Emp ID': 782805,
     'Month Name of Joining': 'November',
     'Last Name': 'Sorenson',
     'Gender': 'M',
     'E Mail': 'santo.sorenson@gmail.com',
     'SSN': '548-99-8083',
     'County': 'Union',
     'State': 'NJ',
     'Region': 'Northe

### Task 3: Generate employee summary and ordering by Gender and Salary

In [20]:
es.search(index="humans", size=5, body={"query": {"match_all": {}}, "sort":[{"Gender.keyword":{"order":"asc"}}, {"Salary":{"order":"asc"}}]})

{'took': 221,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': [{'_index': 'humans',
    '_type': '_doc',
    '_id': 'h-y6DnUBFUuBiHlmt2vx',
    '_score': None,
    '_source': {'Emp ID': 506893,
     'Month Name of Joining': 'June',
     'Last Name': 'Cumberbatch',
     'Gender': 'F',
     'E Mail': 'ariana.cumberbatch@msn.com',
     'SSN': '057-02-5798',
     'County': 'Evangeline',
     'State': 'LA',
     'Region': 'South',
     'City': 'Saint Landry',
     'Zip': 71367,
     'Salary': 40000,
     'Salary Hike': 4},
    'sort': ['F', 40000]},
   {'_index': 'humans',
    '_type': '_doc',
    '_id': '7QS8DnUBFUuBiHlmQM0j',
    '_score': None,
    '_source': {'Emp ID': 374296,
     'Month Name of Joining': 'March',
     'Last Name': 'Owen',
     'Gender': 'F',
     'E Mail': 'clyde.owen@gmail.com',
     'SSN': '366-39-8276',
     'County': 'Shawano',
     'St

### Task 4: Summerize the number of employee joined and hikes granted based on month

In [21]:
es.search(index="humans", size=0, body={"query": {"match_all": {}}, "aggs":{"No. of Employees joined in particular month" : {"terms":{"field": "Month Name of Joining.keyword"}}}})

{'took': 98,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'No. of Employees joined in particular month': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 766909,
   'buckets': [{'key': 'August', 'doc_count': 468295},
    {'key': 'July', 'doc_count': 450981},
    {'key': 'May', 'doc_count': 434522},
    {'key': 'June', 'doc_count': 427424},
    {'key': 'March', 'doc_count': 422987},
    {'key': 'April', 'doc_count': 414484},
    {'key': 'January', 'doc_count': 413247},
    {'key': 'December', 'doc_count': 408721},
    {'key': 'October', 'doc_count': 400048},
    {'key': 'November', 'doc_count': 392382}]}}}

In [22]:
es.search(index="humans", size=0, body={"query": {"match_all": {}}, "aggs":{"No. of Hikes granted per Month": {"cardinality": {"field": "Salary Hike"}}}})

{'took': 194,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'No. of Hikes granted per Month': {'value': 31}}}

### Task 5: Generate employee summary and ordering by Salary

In [23]:
es.search(index="humans", size=5, body={"query":{"match_all":{}}, "sort":[{"Salary":{"order":"asc"}}]})

{'took': 26,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': [{'_index': 'humans',
    '_type': '_doc',
    '_id': 'Le-6DnUBFUuBiHlm7-A2',
    '_score': None,
    '_source': {'Emp ID': 680938,
     'Month Name of Joining': 'November',
     'Last Name': 'Hillyard',
     'Gender': 'M',
     'E Mail': 'rosario.hillyard@gmail.com',
     'SSN': '307-37-6816',
     'County': 'Somerset',
     'State': 'PA',
     'Region': 'Northeast',
     'City': 'Tire Hill',
     'Zip': 15959,
     'Salary': 40000,
     'Salary Hike': 30},
    'sort': [40000]},
   {'_index': 'humans',
    '_type': '_doc',
    '_id': 'h-y6DnUBFUuBiHlmt2vx',
    '_score': None,
    '_source': {'Emp ID': 506893,
     'Month Name of Joining': 'June',
     'Last Name': 'Cumberbatch',
     'Gender': 'F',
     'E Mail': 'ariana.cumberbatch@msn.com',
     'SSN': '057-02-5798',
     'County': 'Evangeline'

### Deleting the index from elasticsearch

In [24]:
es.indices.delete(index='humans')

{'acknowledged': True}

### Stopping the SparkSession

In [25]:
spark.stop()
print("Successfully stopped SparkSession!")

Successfully stopped SparkSession!
